# 재귀와 패턴 매칭

재귀와 패턴 매칭을 간단한 예제를 통해 설명한다. 
보다 복잡한 예제와 설명은 강의를 진행하면서 차차 이루어질 것이다.

## 재귀

**재귀**(recursion)의 사전적 정의는 되풀이 또는 순환반복 등을 의미한다. 
그리고 **재귀 함수**는 재귀로 정의된 함수를 가리킨다.

그런데 재귀로 정의된 함수란 무엇일까?
다음 예제를 살펴보자. 

In [1]:
nonStoppingFtn x = nonStoppingFtn (x + 1)

`nonStoppingFtn` 함수의 본체에 `nonStoppingFtn` 이 사용된다.
이렇듯 함수를 정의할 때 함수의 본체에 자신이 사용되는 것을 재귀라 부른다.
즉, `nonStoppingFtn` 함수처럼 재귀로 정의된 함수를 재귀함수라 부른다. 

사실 `nonStoppingFtn` 함수는 인위적으로 구현된 함수이다. 
Coq, Agda와 같은 증명보조기(proof assistant)가 사용하는 프로그래밍 언어에서는
`nonStoppingFtn` 같은 함수는 애초부터 정의를 받아들이지 않는다.

**주의:** "정의를 받아들이지 않는다" 는 표현을 여기서는 일종의 문법 오류 정도로 이해하면 된다.

반면에 이전에 살펴 본 `sum_`, `qsort_`, `actionSeqn` 함수 또한 모두 재귀 함수들이다. 
하지만 이들은 `nonStoppingFtn`과 다른 종류의 재귀를 사용하는데,
바로 패턴 매칭과 함께 재귀가 사용된다. 

하스켈과 같은 함수형 프로그래밍 언어에서 재귀와 패턴 매칭은 매우 중요한 요소이며 서로 긴밀히 공존하는 관계이다.
여기서는 패턴 매칭과 재귀의 긴밀한 관계를 알아보기 위해 `sum_` 함수의 정의를 분석한다.

## 패턴 매칭과 재귀

하스켈에서 `sum_` 함수의 정의는 다음과 같다.

In [2]:
sum_ :: [Integer] -> Integer
sum_ []= 0
sum_ (n:ns) = n + sum_ ns

Line 2: Use foldr
Found:
sum_ [] = 0
sum_ (n : ns) = n + sum_ ns
Why not:
sum_ ns = foldr (+) 0 ns

위 정의는 다음을 말하고 있다. 

* 공리스트인 경우: 모든 항목들의 합은 0이다.
* 리스트의 첫째 항목이 `n`이고 나머지 항목들로 구성된 리스트가 `ns`인 경우: 
    모든 항목들의 합은 `n`과 나머지 항목들의 합을 더한 값이다. 
    
좀 더 수학적으로 표현하면 다음과 같다.

$$
\text{sum}_{-}(\text{xs})= 
\begin{cases}
0,  & \text{if}\;\; \text{xs = []}\\
\text{n} + \text{sum}_{-}(\text{ns}), &  \text{if}\;\; \text{xs = n:ns}
\end{cases}
$$

즉, `sum_` 인자의 형태를 이용하면서 동시에 재귀를 이용하여 정의되었다. 
그런데 하스켈 인터프리터는 리스트의 형태(패턴)를 어떻게 파악할까? 

파이썬, 자바 등의 언어는 위와 같은 형식의 정의를 지원하지 않는다. 
파이썬의 경우 `sum_` 함수는 보통 다음과 같이 정의한다. 

```python
def sum_(xs):
    if len(xs) == 0:
        return 0
    else:
        return xs[0] + sum_(xs[1:])
```

즉, 인자의 형태를 보고 판단하는 것이 아니라 인자로 들어온 값의 길이에 따라
처리 방식을 달리한다.
예를 들어, 길이가 0이 아니면, 인자가 하나 이상일 것이기 때문에 첫째 항목과 나머지로
구분할 수 있다고 **가정**하고, 그 가정에 맞춘 계산을 지정한다. 

다시 말해, 파이썬, 자바 등의 명령형 프로그래밍 언어는 인자가 어떤 형태인지 파악하지 않거나 못한다. 
아마도 후자일 것이며, 이유는 자료형을 구현할 때,
정의된 자료형을 구조적으로 파악할 수 있는 도구를 함께 제공하지 못하는 방식을 사용하기 때문일 것이다. 

반면에 하스켈 등의 함수형 프로그래밍 언어는
애초부터 구조적으로 파악할 수 있는 방식으로  유형을 정의한다.

재귀적으로 유형을 정의하는 방법을 예를 통해 설명하면서 
재귀와 패턴 매칭의 연관성을 밑바닥부터 자세히 알아본다. 
재귀적 유형에 대한 자세한 설명은 이후에 다룰 것이다.

## 재귀적 유형 예제

유형을 선언하는 여러 방식이 있다. 
여기서는 정수 리스트, 즉 정수들로 구성된 리스트들의 유형을 재귀적으로 선언하면서 재귀적 유형 선언 방식을 소개한다. 

재귀적 유형을 선언하려면 먼저 `data` 키워드를 사용한다.
또한 선언되는 유형의 값을 생성하는 **구성자**(constructor)들을 한 개 이상 지정한다.
구성자들 사이의 구분은 **파이프**라 불리는 작대기 모양의 기호(`'|'`)에 의해 이루어진다. 
엄밀히 말해 구성자는 모두 함수이며, 해당 유형의 값을 생성하는 도구 역할을 수행한다.

```haskell
data TypeName =   Constructor_1 ArgType_11 ... ArgType_1N_1
                | Constructor_2 ArgType_21 ... ArgType_2N_2
                ...
                | Constructor_n ArgType_n1 ... ArgType_nN_n
```

**주의:** 
* 유형과 구성자 이름 모두 대문자로 시작해야 한다.
* 만약에 `kN_k`가 0이면 해당 구성자는 인자를 받지 않는 상수값이 된다.

#### 참고사항

재귀적 유형(recursive types)을 **귀납적 유형**(inductive types)이라 부르기도 한다.
예를 들어, 많은 수리논리 전공서적과 Coq 증명보조기에서 귀납적 유형이란 표현을 사용하며,
수학 증명을 할 때 귀납적 증명이란 표현을 사용한다.
반면에 재귀란 표현은 함수를 재귀적으로 정의할 때, 즉, 재귀 함수에 대해서만 사용한다. 

하지만 '재귀적'과 '귀납적' 두 표현에 대한 엄밀한 구분은 경우에 따라 불가능하다.
따라서 여기서는 두 개념을 구분하지 않고, 재귀 표현으로 통일해서 사용한다. 

### 정수 리스트

구성자를 지정하려면 선언되는 유형들의 형태(패턴)을 먼저 파악해야 하는데,
값들이 어떤 패턴을 갖는가를 알려면 해당 유형을 갖는 값들을 생성하는 방법을 파악해야 한다. 
예를 들어, 정수 리스트는 다음 두 가지 방식 중 하나로 생성할 수 있다.

* 아무런 항목도 포함하지 않는 리스트, 즉 공리스트 `[]`를 생성하거나,
* 임의의 정수 `n`과 임의의 정수 리스트 `ns`를 이용하여 새로운 정수 리스트 `n:ns`를 생성한다.

따라서 두 개의 구성자를 필요해야 한다.

* 공리스트를 생성하는 구성자
    * 이 구성자는 아무 것도 이용하지 않으면서 공리스트를 생성해야 한다. 
    * 이런 경우 구성자는 상수(constant)와 같으며, 구성자 자체를 공리스트로 간주한다.


* 하나의 정수와 하나의 정수 리스트를 이용하여 새로운 정수 리스트를 생성하는 구성자
    * 이 구성자는 두 개의 인자를 받는다.
    * 첫째 인자: 정수
    * 둘째 인자: 정수 리스트
    
구성자의 이름은 임의로 지정할 수 있다. 
리스트의 경우 공리스트는 `Nil`, 정수와 리스트를 이용하여 새로운 리스트를 생성하는 구성자는 `Cons`로
관행적으로 부른다. 
따라서 정수 리스트 유형의 이름을 `IntList`로 할 경우 
다음처럼 `IntList`를 선언한다. 

In [1]:
data IntList =   Nil 
               | Cons Integer IntList

`IntList`에 사용된 두 개의 구성자의 역할을 다시 한 번 확인하면 다음과 같다.

* `Nil`: 인자를 받지 않는 함수, 즉, 하나의 상수(constant)이며, 
    여기서는 공리스트를 가리킨다.
* `Cons`: 인자 두 개를 받는 함수이며, 여기서는 하나의 정수와 다른 정수 리스트를 받아
    새로운 정수 리스트를 만드는 함수를 가리킨다. 

`IntList` 유형이 재귀적으로 선언되었음에 주의한다. 
실제로 `Cons`의 둘째 인자가 `IntList` 유형의 값이어야 한다. 

#### `IntList`의 진짜 의미

위 정의가 실제로 의미하는 바는 다음과 같다. 

`IntList` 유형을 갖는 값은 아래 두 가지 방식 중 하나를 이용해서만 생성할 수 있으며,
다른 방식은 허용되지 않는다.
* 첫째 방식: `Nil` 자체가 `IntList` 유형을 갖는 값이다
* 둘째 방식: `n`이 `Integer` 유형의 값, 즉, 하나의 정수이고,
    `ns`가 (이미) `IntList`의 유형의 값이면 
    `Cons n ns` 또한 `IntList` 유형의 값이다.

**참고:** Cons는 construct의 줄임말이며,
    프로그래밍언어론 분야에서 많이 사용되는 표현이다.
    `Cons n ns`가 `n`과 `ns`를 이용하여 새로운 리스트를 생성한다는 의미와 일맥상통한다.

### 정수 리스트 예제

`IntList` 유형을 갖는 값들은 예를 들어 다음과 같다.

* 공리스트

In [2]:
:t Nil

Nil :: IntList

* `[1]`에 해당하는 값

In [3]:
:t (Cons 1 Nil)

(Cons 1 Nil) :: IntList

* `[1, 2]`에 해당하는 값

In [4]:
:t (Cons 1 (Cons 2 Nil))

(Cons 1 (Cons 2 Nil)) :: IntList

* `[1, 2, 3]`에 해당하는 값

In [5]:
:t (Cons 1 (Cons 2 (Cons 3 Nil)))

(Cons 1 (Cons 2 (Cons 3 Nil))) :: IntList

#### 주의사항: 화면 출력 오류

다음과 같이 입력하고 실행하면 오류가 발생한다. 

In [6]:
Cons 1 (Cons 2 (Cons 3 Nil))

: 

위 오류가 발생하는 이유는 `IntList` 유형의 값을 화면에 출력하는 기능이 지원되지 않기 때문이다.
이를 교정하는 방법은 `IntList`를 `Show` 클래스의 인스턴스로 지정하는 것이다.
일단 여기서는 필요하지 않기에 다음에 다루기로 하고 일단 넘어가도록 한다.

**참고사항:** 파이썬 클래스를 잘 안다면 `__repr__` 메소드와 연관시켜 이해할 수 있다. 
하지만 여기서 자세한 설명은 하지 않는다.

## `IntList`와 패턴 매칭

`IntList` 유형은 정수로 이루어진 리스트들의 유형 즉, `[Integer]`와 동형(isomorphic)이다.
두 유형의 동형성을 몇 가지 예를 통해 보이고자 한다. 

### 항목들의 합 구하기

정수 리스트에 포함된 항목들의 합을 계산하는 `sum_` 함수를 `IntList` 유형에 대해 
동일하게 작동하는 함수 `intSum`을 구현해보자.

`intSum`은 인자로 들어오는 `IntList` 유형의 값들이 갖는 형태(패턴)를 이용하여 정의된다.
즉, 인자들의 패턴(형태)에 따른 값을 매칭(지정)하는 방식으로 `intSum`을 선언한다.

그런데 `IntList` 유형의 값은 다음 두 가지 형태 중 하나를 갖는다.

* `Nil'` 또는
* `Cons n ns`.
    * 단 `n`은 정수이고 `ns`는 `IntList` 유형을 갖는다.
    
따라서 `intSum`을 다음과 같이 정의될 수 있다.

In [7]:
intSum :: IntList -> Integer
intSum Nil = 0
intSum (Cons n ns) = n + intSum ns

위 정의는 재귀적으로 선언되었음에 주의하라. 
또한 `sum_` 함수의 정의와 사실상 동일하다. 

* `Nil`: `[]`에 대응
* `Cons n ns`: `(n:ns)`에 대응

다만, 정수 리스트들의 유형을 `[Integer]`에서 `IntList`로 바뀐 것 뿐이다.

#### `intSum` 활용 예제

* 공리스트 항목의 합은 0

In [8]:
intSum Nil

0

* `[1]` 항목의 합은 1

In [9]:
intSum (Cons 1 Nil)

1

* `[1, 2]` 항목의 합은 3

In [10]:
intSum (Cons 1 (Cons 2 Nil))

3

* `[1, 2, 3]` 항목의 합은 6

In [11]:
intSum (Cons 1 (Cons 2 (Cons 3 Nil)))

6

### 리스트 관련 함수 직접 구현하기

하스켈의 표준 라이브러리에 리스트를 다루는 다양한 도구들이 함수로 정의되어 있다.

* `head`: 공리스트가 아닌 리스트의 첫째 항목 계산하기

In [12]:
head [1..10]

1

* `tail`: 공리스트가 아닌 리스트에서 첫째 항목을 제외한 나머지 항목들의 리스트 계산하기

In [13]:
tail [1..10]

[2,3,4,5,6,7,8,9,10]

* `last`: 공리스트가 아닌 리스트의 마지막 항목 계산하기

In [14]:
last [1..10]

10

* `init`: 공리스트가 아닌 리스트에서 마지막 항목을 제외한 나머지 항목들의 리스트 계산하기

In [15]:
init [1..10]

[1,2,3,4,5,6,7,8,9]

* `!!`: 리스트의 n번째 항목 계산하기. (n은 0부터 시작)

In [16]:
[1..10] !! 3

4

**주의사항:** `!!`는 사칙연산자들처럼 두 인자 사이에 위치하는 중위 함수(infix function)이다.
이처럼 인자 두 개를 받는 함수는 중위 표기법(infix notation)을 적용할 수 있다.
이에 대해서 나중에 자세히 다룬다.

* `take`: 리스트의 첫 n개 항목으로 구성된 리스트 계산하기

In [17]:
take 4 [1..10]

[1,2,3,4]

* `drop`: 리스트에서 첫 n개 항목을 삭제하기

In [18]:
drop 4 [1..10]

[5,6,7,8,9,10]

* `length`: 리스트의 길이 계산하기

In [19]:
length [1..10]

10

* `sum`: 리스트의 모든 항목 더하기

In [20]:
sum [1..10]

55

* `product`: 리스트의 모든 항목 곱하기

In [21]:
product [1..10]

3628800

* `++`: 리스트 두 개 이어붙이기

In [22]:
[1..4] ++ [5..10]

[1,2,3,4,5,6,7,8,9,10]

`++` 또한 중위 함수임에 주의한다.

* `reverse`: 항목들의 순서를 뒤집은 리스트 생성하기

In [23]:
reverse [1..10]

[10,9,8,7,6,5,4,3,2,1]

이 중에 `head`와 `tail`를 `IntList` 유형에 대해 작동하는 함수로 구현해 보자.
역시 패턴 매칭을 사용하면 쉽게 구현할 수 있다.

In [24]:
intHead :: IntList -> Integer
intHead (Cons n ns) = n

In [25]:
intTail :: IntList -> IntList
intTail (Cons n ns) = ns

`intHead`와 `intTail`은 공리스트에 대해서는 전혀 정의되어 있지 않다.
따라서 공리스트를 인자로 사용하면 오류를 발생시킨다. 

In [26]:
intHead Nil

: 

In [29]:
intTail Nil

: 

오류를 발생시키지 않기 위해 공리스트 인자에 대해 오류처리를 할 수 있다.
실제로 `head`, `tail`은 공리스트 인자에 대해 다음과 같이 작동한다. 

In [30]:
head []

: 

In [31]:
tail []

: 

하지만 오류처리는 여기서 다룰 수 없다. 다음 기회에 배우도록 하자.

## 연습문제

`IntList` 유형에 대해 
`last`, `init`, `!!`, `take`, `drop`, `length`, `product`, `++`, `reverse` 처럼
작동하는 함수들을 패턴 매칭을 이용하여 구현하라.
